In [3]:
#
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import os
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [4]:
animaldf = pd.read_csv("animal_info.csv", sep = ";")
animaldf.columns = [col.replace(" ", "_").lower().replace("-", "") for col in animaldf.columns]
fresh_dates = animaldf[["animal_number", "last_calving_date"]]
fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')
fresh_dates = fresh_dates[~fresh_dates.fresh_date.isnull()]
fresh_dates = fresh_dates.drop("last_calving_date", axis = 1)
fresh_dates = fresh_dates[(~fresh_dates.animal_number.isna())]

In [5]:
df = pd.read_csv(r"more_reports\oneyear_data.csv", sep = ";")
df.columns = [col.replace(" ", "_").lower().replace("-", "") for col in df.columns]
df["date"] = pd.to_datetime(df["date"], format = '%m/%d/%Y')
merged_df =  pd.merge(fresh_dates, df,  left_on="animal_number", right_on = "animal_number")

merged_df["dim"] = (merged_df["date"] - merged_df["fresh_date"]).dt.days
df2 = merged_df[merged_df.dim >= 0]

# Daily incomplete milkings

## current lactation

In [6]:
inc_df = df2[['animal_number', 'date', 'incomplete', 'dim']]
inc_dfg = inc_df.groupby(["animal_number", "date", "dim"]).agg("sum").reset_index()
inc_dfg["dinc"]  = [1 if val >0 else 0 for val in inc_dfg.incomplete]

inc_dfg = inc_dfg.groupby(["animal_number"]).agg({"dinc":"sum", "dim": "max"}).reset_index()
inc_dfg = inc_dfg.rename(columns = {"dinc":"dinc_cl"})

inc_dfg["dinc_dim_cl"] = (inc_dfg["dinc_cl"]/inc_dfg["dim"])*100

inc1 = inc_dfg

##  7 days

In [7]:
inc_df = df2[(df2.date >= '2023-11-10') & (df2.date < '2023-11-17')]
inc_df = inc_df[['animal_number', 'date', 'incomplete', 'dim']]
inc_dfg = inc_df.groupby(["animal_number", "date", "dim"]).agg("sum").reset_index()
inc_dfg["dinc"]  = [1 if val >0 else 0 for val in inc_dfg.incomplete]

inc_dfg = inc_dfg.groupby(["animal_number"]).agg({"dinc":"sum"}).reset_index()
inc_dfg = inc_dfg.rename(columns = {"dinc":"dinc_7d"})

inc_dfg["dinc_dim_7d"] = (inc_dfg["dinc_7d"]/7)*100

inc2 = inc_dfg

# Visit incomplete milkings

## current lactation

In [8]:
inc_df = df2[['animal_number', 'date', 'incomplete', 'dim']]
inc_dfg = inc_df.groupby(["animal_number"]).agg({"incomplete":"sum", "date": "count"}).reset_index()
inc_dfg = inc_dfg.rename(columns = {"incomplete":"vinc_cl", "date": "visits_cl"})

inc_dfg["vinc_nv_cl"] = (inc_dfg["vinc_cl"]/inc_dfg["visits_cl"])*100
inc3 = inc_dfg

## 7  days

In [9]:
inc_df = df2[(df2.date >= '2023-11-10') & (df2.date < '2023-11-17')]
inc_df = inc_df[['animal_number', 'date', 'incomplete', 'dim']]
inc_dfg = inc_df.groupby(["animal_number"]).agg({"incomplete":"sum", "date": "count"}).reset_index()
inc_dfg = inc_dfg.rename(columns = {"incomplete":"vinc_7d", "date": "visits_7d"})

inc_dfg["vinc_nv_7d"] = (inc_dfg["vinc_7d"]/inc_dfg["visits_7d"])*100
inc4 = inc_dfg

# Merge and save 

In [10]:
integrated_df = inc1.merge(inc2, how='left', on = "animal_number").merge(inc3, how='left', on = "animal_number").merge(inc4, how='left', on = "animal_number")
integrated_df.columns = [col.replace("animal_number", "cow") for col in integrated_df.columns]
integrated_df.to_csv(r"data_out\inc_integrated.csv", index = False)